In [1]:
# Dependencies
import pandas as pd
import numpy as np
import scipy.stats as st
import matplotlib.pyplot as plt
import requests
import json
import gmaps
from config import g_key
gmaps.configure(api_key=g_key)

In [2]:
# import filtered shark csv file
filename = "Resource/Shark Data WA 2016-2021 (filtered).xls"
shark_data = pd.read_excel(filename)
shark_data.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Name,Sex,...,Fatal (Y/N),Time,Species,Investigator or Source,pdf,href formula,href,Case Number.1,Case Number.2,original order
0,2016.12.06,06-Dec-2016,2016.0,Provoked,AUSTRALIA,New South Wales,Merimbula,Surf fishing,Jesse Phillips,M,...,N,Night,"Wobbegong shark, 2m",Bay 93.9 Geelong,2016.12.06-Phillips.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.12.06,2016.12.06,6125
1,2020.11.02,02-Nov-2020,2020.0,Unprovoked,AUSTRALIA,New South Wales,"Town Beach, Port Macquarie",Surfing,male,M,...,N,Dawn,NaN,"B. Myatt, GSAF",2020.11.02-PortMacquarie.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.11.02,2020.11.02,6593
2,2017.09.10.a,10-Sep-2017,2017.0,Unprovoked,AUSTRALIA,New South Wales,Iluka Beach,Surfing,Abe McGrath,M,...,N,Dawn,"White shark, 3 to 3.5m","B. Myatt, GSAF",2017.09.10.a-McGrath.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.09.10.a,2017.09.10.a,6234
3,2019.02.04.a,04-Feb-2019,2019.0,Unprovoked,AUSTRALIA,New South Wales,Urunga area,Surfing,Matt Gallagher,M,...,N,Dawn,NaN,"K. McMurray, TrackingSharks.com",2019.02.04.a-Gallagher.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2019.02.04.a,2019.02.04.a,6400
4,2019.02.17,17-Feb-2019,2019.0,Unprovoked,AUSTRALIA,New South Wales,Belongil Bay,Surfing,Sam Edwardes,M,...,N,Dawn,NaN,"B. Myatt, GSAF",2019.02.17-Edwardes.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2019.02.17,2019.02.17,6404


In [3]:
shark_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 163 entries, 0 to 162
Data columns (total 22 columns):
 #   Column                  Non-Null Count  Dtype  
---  ------                  --------------  -----  
 0   Case Number             163 non-null    object 
 1   Date                    163 non-null    object 
 2   Year                    162 non-null    float64
 3   Type                    163 non-null    object 
 4   Country                 163 non-null    object 
 5   Area                    163 non-null    object 
 6   Location                161 non-null    object 
 7   Activity                156 non-null    object 
 8   Name                    160 non-null    object 
 9   Sex                     158 non-null    object 
 10  Age                     118 non-null    object 
 11  Injury                  162 non-null    object 
 12  Fatal (Y/N)             155 non-null    object 
 13  Time                    163 non-null    object 
 14  Species                 106 non-null    ob

In [4]:
shark_data['Area'].unique()

array(['New South Wales', 'Queensland', 'Western Australia',
       'New South Wales ', 'Victoria', 'Tasmania', 'South Australia',
       'Westerm Australia', 'Northern Territory '], dtype=object)

In [5]:
# clean up misspelled State
shark_data['Area'] = shark_data['Area'].replace({'Westerm Australia':'Western Australia'})
shark_data['Area'] = shark_data['Area'].replace({'New South Wales ':'New South Wales'})

In [6]:
# remove unwanted columns
shark_df = shark_data.drop(['Name','Investigator or Source'], axis = 1)


In [7]:
shark_df['Location'].nunique()

148

In [8]:
# match location to lat and long coordinates, using Google Maps API

# create columns to hold data
shark_df['lat']=""
shark_df['lng']=""

# Build URL using the Google Maps API
base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

params = {"key": g_key, "inputtype": "textquery", "fields":"geometry"}

for index, row in shark_df.iterrows():
    # get extra parameters
    params['input'] = (f'{row["Location"]}, {row["Area"]}')
    
    # Run request
    print(f"Retrieving Results for Index {index}: {row['Location']}.")
    response = requests.get(base_url, params=params)
    results = response.json()

    # Extract lat/lng
    try:
        shark_df.loc[index, 'lat'] = results['candidates'][0]['geometry']['location']['lat']
        shark_df.loc[index, 'lng'] = results['candidates'][0]['geometry']['location']['lng'] 
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

    


Retrieving Results for Index 0: Merimbula.
Retrieving Results for Index 1: Town Beach, Port Macquarie.
Retrieving Results for Index 2: Iluka Beach.
Retrieving Results for Index 3: Urunga area.
Retrieving Results for Index 4: Belongil Bay.
Retrieving Results for Index 5: Shelly Beach, Ballina.
Retrieving Results for Index 6: Moffat Beach.
Retrieving Results for Index 7: Cable Beach.
Retrieving Results for Index 8: Samurai Beach.
Retrieving Results for Index 9: Lennox Head.
Retrieving Results for Index 10: Nambucca.
Retrieving Results for Index 11: Winkipop.
Retrieving Results for Index 12: Broken Head Beach.
Retrieving Results for Index 13: Blackwall Reach.
Retrieving Results for Index 14: Killick Creek near Crescent Head.
Retrieving Results for Index 15: Windang Beach.
Retrieving Results for Index 16: Pelican Waters, Caloundra.
Retrieving Results for Index 17: Cobblestones, Margaret River Area.
Retrieving Results for Index 18: Cone Bay.
Retrieving Results for Index 19: Point Casuarina,

In [9]:
shark_df.head()

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,...,Time,Species,pdf,href formula,href,Case Number.1,Case Number.2,original order,lat,lng
0,2016.12.06,06-Dec-2016,2016.0,Provoked,AUSTRALIA,New South Wales,Merimbula,Surf fishing,M,20,...,Night,"Wobbegong shark, 2m",2016.12.06-Phillips.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.12.06,2016.12.06,6125,-36.8875,149.906
1,2020.11.02,02-Nov-2020,2020.0,Unprovoked,AUSTRALIA,New South Wales,"Town Beach, Port Macquarie",Surfing,M,13,...,Dawn,NaN,2020.11.02-PortMacquarie.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.11.02,2020.11.02,6593,-31.4295,152.919
2,2017.09.10.a,10-Sep-2017,2017.0,Unprovoked,AUSTRALIA,New South Wales,Iluka Beach,Surfing,M,35,...,Dawn,"White shark, 3 to 3.5m",2017.09.10.a-McGrath.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2017.09.10.a,2017.09.10.a,6234,-29.412,153.365
3,2019.02.04.a,04-Feb-2019,2019.0,Unprovoked,AUSTRALIA,New South Wales,Urunga area,Surfing,M,50,...,Dawn,NaN,2019.02.04.a-Gallagher.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2019.02.04.a,2019.02.04.a,6400,-30.5,153.017
4,2019.02.17,17-Feb-2019,2019.0,Unprovoked,AUSTRALIA,New South Wales,Belongil Bay,Surfing,M,41,...,Dawn,NaN,2019.02.17-Edwardes.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2019.02.17,2019.02.17,6404,-28.6337,153.6


In [10]:
# find record where location is not given
no_loc = shark_df.loc[shark_df['Location'].isna()]
print(no_loc)

# this record is not in WA, so will not affect study too much, so will be dropped from df
shark_df = shark_df.loc[shark_df['Location'].notna()]





      Case Number                  Date    Year      Type    Country  \
43   2018.12.15.b           15-Dec-2018  2018.0  Provoked  AUSTRALIA   
154  2017.01.08.R  Reported 08-Jan-2017     NaN   Invalid  AUSTRALIA   

                Area Location      Activity Sex  Age  ...     Time  \
43   New South Wales      NaN           NaN    M  48  ...      Day   
154       Queensland      NaN  Spearfishing    M  35  ...  Unknown   

            Species                            pdf  \
43   Wobbegong shark      2018.12.15-Wobbegong.pdf   
154       Bull shark  2017.01.08.R-KerryDaniel.pdf   

                                          href formula  \
43   http://sharkattackfile.net/spreadsheets/pdf_di...   
154  http://sharkattackfile.net/spreadsheets/pdf_di...   

                                                  href Case Number.1  \
43   http://sharkattackfile.net/spreadsheets/pdf_di...  2018.12.15.b   
154  http://sharkattackfile.net/spreadsheets/pdf_di...  2017.01.08.R   

    Case Number.2

In [11]:
# convert lat and lng results to numbers
shark_df['lat'] = pd.to_numeric(shark_df['lat'])
shark_df['lng'] = pd.to_numeric(shark_df['lng'])

In [12]:
empty = shark_df.isna()
empty.sum()

Case Number        0
Date               0
Year               0
Type               0
Country            0
Area               0
Location           0
Activity           6
Sex                5
Age               45
Injury             1
Fatal (Y/N)        7
Time               0
Species           57
pdf                0
href formula       0
href               0
Case Number.1      0
Case Number.2      0
original order     0
lat                2
lng                2
dtype: int64

In [13]:
# find rows with empty locations
shark_df_empty_loc = shark_df.loc[shark_df['lat'].isna()]
shark_df_empty_loc

,Case Number,Date,Year,Type,Country,Area,Location,Activity,Sex,Age,...,Time,Species,pdf,href formula,href,Case Number.1,Case Number.2,original order,lat,lng
40,2020.10.09,09-Oct-2020,2020.0,Unprovoked,AUSTRALIA,Western Australia,Kelp Beds Beach (Kelpies),Surfing,M,52,...,Day,4m shark,2020.10.09-Sharpe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2020.10.09,2020.10.09,6586,NaN,NaN
117,2016.07.20,20-Jul-2016,2016.0,Provoked,AUSTRALIA,Queensland,"20 k off The Spit, off the Gold Coast",Fishing,M,31,...,Dusk,"reef shark, 1m",2016.07.20-Burck.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...,http://sharkattackfile.net/spreadsheets/pdf_di...,2016.07.20,2016.07.20,6079,NaN,NaN


In [14]:
# these rows are important to the findings, so will need to find location data for them

# Build URL using the Google Maps API

base_url = "https://maps.googleapis.com/maps/api/place/findplacefromtext/json"

params = {"key": g_key, "inputtype": "textquery", "fields":"geometry"}

# Sam's Creek Area, Point Samson WA, case number 2017.09.10.b
params['input'] = "Point Samson WA"
response = requests.get(base_url, params=params)
results = response.json()
shark_df.loc[shark_df['Case Number'] == "2017.09.10.b",'lat'] = results['candidates'][0]['geometry']['location']['lat']
shark_df.loc[shark_df['Case Number'] == "2017.09.10.b",'lng'] = results['candidates'][0]['geometry']['location']['lng']

# bundegi sanctuary zone, case number 2016.12.24
params['input'] = "bundegi sanctuary zone"
response = requests.get(base_url, params=params)
results = response.json()
shark_df.loc[shark_df['Case Number'] == "2016.12.24",'lat'] = results['candidates'][0]['geometry']['location']['lat']
shark_df.loc[shark_df['Case Number'] == "2016.12.24",'lng'] = results['candidates'][0]['geometry']['location']['lng']

# The Spit Gold Coast, Seaworld Drive, Main Beach QLD, case number 2016.07.20
params['input'] = "The Spit Gold Coast, Seaworld Drive, Main Beach QLD"
response = requests.get(base_url, params=params)
results = response.json()
shark_df.loc[shark_df['Case Number'] == "2016.07.20",'lat'] = results['candidates'][0]['geometry']['location']['lat']
shark_df.loc[shark_df['Case Number'] == "2016.07.20",'lng'] = results['candidates'][0]['geometry']['location']['lng']

# Kelp Beds surf break, Wylie Bay, Esperance, case number 2020.10.09
params['input'] = "Wylie Bay, Esperance"
response = requests.get(base_url, params=params)
results = response.json()
shark_df.loc[shark_df['Case Number'] == "2020.10.09",'lat'] = results['candidates'][0]['geometry']['location']['lat']
shark_df.loc[shark_df['Case Number'] == "2020.10.09",'lng'] = results['candidates'][0]['geometry']['location']['lng']




In [15]:
# export data
shark_df.to_csv("Resource/shark_geo.csv")